Patrón Mediator

Se realiza la implementación del patrón de diseño mediator por medio de las clases Customer, Cook en este caso la clase Cashier es el mediator entonces las solicitudes no pasan directamente entre los objetos Customer y Cook si no que las maneja el mediator Cashier, con el que nuestras clases que deben trabajar juntas quedan totalmente desacopladas

In [5]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from traceback import print_tb


@dataclass
class Order:
   order: str
   name: str
   is_paid: bool = False
   is_prepared: bool = False


class ICashierMediator(ABC):

    @abstractmethod
    def make_order(self, order_: Order) -> None:
        pass
    @abstractmethod
    def pay_order(self) -> None:
        pass
    @abstractmethod
    def deliver_order(self, order_: Order) -> None:
        pass

class Customer:
    def __init__(self, name: str, cashier_mediator: ICashierMediator = None) -> None:
        self.cashier_mediator = cashier_mediator
        self.name = name

    def order_food(self, order_: Order) -> None:
        self.cashier_mediator.make_order(order_)
        print("generando orden")

    def pay_food(self) -> None:
        self.cashier_mediator.pay_order()
        print("pagando orden")

    def get_order_food(self, order_: Order) -> None:
        self.cashier_mediator.deliver_order(order_)


class Cook:
    def __init__(self, cashier_mediator: ICashierMediator = None) -> None:
        self.list_orders: list[Order] =  []
        self.cashier_mediator = cashier_mediator

    def add_order(self, order_: Order) -> None:
        self.list_orders.append(order_)

    def prepare_food(self, order_: Order) -> None:
        for order_data in self.list_orders:
            if order_data.name == order_.name and order_data.is_paid:
                order_data.is_prepared = True
                print(f"orden preparada :: {order_data.order} para el cliente {order_data.name}")

    def deliver_food(self, order_: Order) -> None:
        self.cashier_mediator.deliver_order(order_)


class CashierMediator(ICashierMediator):
    def __init__(self, customer_: Customer, cook_: Cook) -> None:
        self.customer_ = customer_
        self.cook_ = cook_

    def make_order(self, order_: Order) -> bool:
        print("creando orden del cliente :: ", self.customer_.name)
        self.cook_.add_order(order_)
        print("Asignando orden del cocinero :: ", order_)
        return True

    def pay_order(self) -> None:
        print("cliente pagando orden")
        print("ordenes asignadas al cook :: ", len(self.cook_.list_orders))
        for order_data in self.cook_.list_orders:
            print("existe order :: ", order_data)
            if order_data.name.lower() == self.customer_.name.lower():
                order_data.is_paid = True
                print("enviar datos al cook para que prepare la comida")
                self.cook_.prepare_food(order_data)

    def deliver_order(self, order_data: Order) -> None:
        for order_data in self.cook_.list_orders:
            if order_data.name.lower() == self.customer_.name.lower() and order_data.is_prepared:
                print(order_data)
                print(f"orden :: {order_data.order} para el cliente {order_data.name} esta lista para entrega")
                self.cook_.list_orders.remove(order_data)





In [6]:
customer: Customer = Customer("juan")
cook: Cook = Cook()
cashier: CashierMediator = CashierMediator(customer, cook)
customer.cashier_mediator = cashier
cook.cashier_mediator = cashier
order: Order = Order("hamburguesa", "juan")

customer.order_food(order)
customer.pay_food()
cook.deliver_food(order)
print(len(cashier.cook_.list_orders))
#customer.get_order_food(Order("hamburguesa", "juan"))


creando orden del cliente ::  juan
Asignando orden del cocinero ::  Order(order='hamburguesa', name='juan', is_paid=False, is_prepared=False)
generando orden
cliente pagando orden
ordenes asignadas al cook ::  1
existe order ::  Order(order='hamburguesa', name='juan', is_paid=False, is_prepared=False)
enviar datos al cook para que prepare la comida
orden preparada :: hamburguesa para el cliente juan
pagando orden
Order(order='hamburguesa', name='juan', is_paid=True, is_prepared=True)
orden :: hamburguesa para el cliente juan esta lista para entrega
0
